# Generation of parameter files for vocal tract simulation and cartographies

In [1]:
import numpy as np
import pyDOE as doe
import matplotlib.pyplot as plt
import h5py

#Direct input 
plt.rcParams['text.latex.preamble']= r"\usepackage{lmodern}"
#Options
params = {'text.usetex' : True,
          'font.size' : 13,
          'font.family' : 'lmodern',
          }
plt.rcParams.update(params) 

%matplotlib qt5

## Generation of the initial DOE : Latin hypercube sampling

We here use the existing library pyDOE to generate the initial design of experiments.

### 2D example : 

In [6]:
N_points = 8
design = doe.lhs(2, N_points, criterion = 'c')
plt.figure(figsize=(3, 3))
#plt.suptitle("Exemple d'échantillonage latin d'une grille 2D avec 8 points")
#plt.subplot(1, 2, 1)
plt.scatter(design[:, 0], design[:, 1], color ='r')
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')
#plt.title('Points centrés')
plt.xlim(0, 1)
plt.ylim(0, 1)

plt.hlines(np.linspace(0, 1, N_points+1)[1:-1], 0, 1, linestyle = '--', color = 'b')
plt.vlines(np.linspace(0, 1, N_points+1)[1:-1], 0, 1, linestyle = '--', color = 'b')


In [7]:
plt.figure(figsize=(3, 3))
#plt.subplot(1, 2, 2)
design = doe.lhs(2, N_points)
plt.scatter(design[:, 0], design[:, 1], color ='r')
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')
#plt.title('Points aléatoires')

plt.hlines(np.linspace(0, 1, N_points+1)[1:-1], 0, 1, linestyle = '--', color = 'b')
plt.vlines(np.linspace(0, 1, N_points+1)[1:-1], 0, 1, linestyle = '--', color = 'b')
plt.xlim(0, 1)
plt.ylim(0, 1)

plt.tight_layout()

### Application to the larynx parameters

In [10]:
#a0s = np.linspace(6e9, 1e10, 10)
#for j, a0 in enumerate(a0s):
      # Fixed parameters

fixed_params = {"rho0":float(1.3),      "L0":float(0.011), "l0":float(2e-3),  "h0":float(1e-4),
      "hr":float(1e-4),       "k0":float(100),   "k1":float(97),    "kappa0":float(3 * 100),
      "kappa1":float(3 * 97), "m":float(2e-4), "S_sub":float(11e-5),
      "S_sup":float(11e-7), "q0":float(0.4),   "w0":float(2 * np.pi * 640), "tdelay":float(1e-3), "trise":float(5e-3), "tmax":float(2),
      "r":float(1.3e-3)}#, "a0":float(a0)}
fs = float(10000)
Te = float(1/fs)
epsilon = 1e-10
solver_params = {"fs":fs, "Te":Te, "epsilon":epsilon}
N_points = 500
carto_params = {"hr":[0 for i in range(N_points)], "P0":[0 for i in range(N_points)], "a0":[0 for i in range(N_points)]}
carto_limits = {"hr":[1e-5, 5e-4], "P0":[0,1000], "a0":[5e9, 1e10]}
carto_scale = {"hr":'lin', 'P0':'lin', "a0":'lin'}
# Generation of the lhs
design = doe.lhs(len(carto_params.keys()), N_points)
i=0
for key, value in carto_params.items():
    if carto_scale[key]=='log':
        carto_params[key] = np.exp(design[:, i]*(np.log(carto_limits[key][1]) - np.log(carto_limits[key][0])) + np.log(carto_limits[key][0]))
    else:
        carto_params[key] = np.array(design[:, i]*(carto_limits[key][1] - carto_limits[key][0]) + carto_limits[key][0])
    i+=1
# Writing in a file
filename  =f'simulations_parameters/CartoP0hra0/a0_{0}.hdf5'
with h5py.File(filename, "w") as f:
    grp_fixed_params = f.create_group("model/fixed_parameters")
    for key, value in fixed_params.items():
        grp_fixed_params.attrs[key] = value
    grp_variable_params = f.create_group("model/variable_parameters")
    for key, value in carto_params.items():
        grp_variable_params.create_dataset(key, data=value)
    grp_solver_params = f.create_group("solver_parameters")
    for key, value in solver_params.items():
        grp_solver_params.attrs[key] = value


In [74]:
# Figure
fig = plt.figure()
ax = fig.add_subplot()#projection='2d')

ax.scatter(carto_params['P0'], carto_params['hr'])
